# Export Promptflow Outputs

This script is used for downloading all the run info needed and merge them together in local environments.


## Prerequisite
Make sure all required libraries are installed.Use command below to install azureml.core:

    pip install azureml-core

In [1]:
import csv
import json
import logging
import requests
from pathlib import Path
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

C:\Users\mogal\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


## Get workspace
Get gonnections to workspace with personal token.

In [2]:
# Get a logger which allows us to log events that occur when running the program.
logger = logging.getLogger("myLogger")
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter("%(asctime)s %(message)s")
ch = logging.StreamHandler()
ch.setFormatter(formatter)
ch.setLevel(logging.DEBUG)
if not logger.handlers:
    logger.addHandler(ch)

# Get the workspace associated with your personal token and get the relavant datastore.
tenant_id = "f64c46b1-d265-46fb-b644-4a4098019d69"
subscription_id = "2f47aff2-af59-42b1-b6d7-524d6f61230c"
resource_group = "first_resource"
workspace_name = "kaggle_essayscoring"

interactive_auth = InteractiveLoginAuthentication(tenant_id=tenant_id)
ws = Workspace.get(workspace_name, subscription_id=subscription_id, resource_group=resource_group, auth=interactive_auth)
region = ws.location
default_datastore = ws.get_default_datastore()

## Define utils
Define functions for downloading and loading run info.

In [3]:
# Get run infor for the given run_id
def get_run_info(run_id):
    logger.info(f"Getting Run Info for Run: {run_id}")
    run = ws.get_run(run_id=run_id)
    display_name = run.display_name
    input_run_id = run.properties.get('azureml.promptflow.input_run_id')
    return display_name, input_run_id

# Get the output asset id for the given run_id and asset_name
def get_output_asset_id(run_id, asset_name):
    logger.info(f"Getting Output Asset Id for Run {run_id}")
    if region == "centraluseuap":
        url = f"https://int.api.azureml-test.ms/history/v1.0/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace_name}/rundata"
    else:
        url = f"https://ml.azure.com/api/{region}/history/v1.0/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace_name}/rundata"
    payload = {
        "runId": run_id,
        "selectRunMetadata": True
    }
    response = requests.post(url, json=payload, headers=ws._auth.get_authentication_header())
    if response.status_code != 200:
        raise Exception(f"Failed to get output asset id for run {run_id} because RunHistory API returned status code {response.status_code}. Response: {response.text}")
    output_asset_id = response.json()["runMetadata"]["outputs"][asset_name]["assetId"]
    return output_asset_id

# Get the asset path for the given asset_id
def get_asset_path(asset_id):
    logger.info(f"Getting Asset Path for Asset Id {asset_id}")
    if region == "centraluseuap":
        url = f"https://int.api.azureml-test.ms/data/v1.0/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace_name}/dataversion/getByAssetId"
    else:
        url = f"https://ml.azure.com/api/{region}/data/v1.0/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace_name}/dataversion/getByAssetId"
    payload = {
        "value": asset_id,
    }
    response = requests.post(url, json=payload, headers=ws._auth.get_authentication_header())
    if response.status_code != 200:
        raise Exception(f"Failed to get asset path for asset id {asset_id} because Data API returned status code {response.status_code}. Response: {response.text}")
    data_uri = response.json()["dataVersion"]["dataUri"]
    relative_path = data_uri.split("/paths/")[-1]
    return relative_path

# Get the flow artifact relative path for the given run_id
def get_flow_artifact_relative_path(run_id):
    logger.info(f"Getting Flow Artifact Relative Path for Run {run_id}")
    try:
        flow_artifact_asset_id = get_output_asset_id(run_id, "debug_info")
        relative_path = get_asset_path(flow_artifact_asset_id)
        relative_path += "flow_artifacts/"
        return relative_path
    except Exception as e:
        logger.warning("`debug_info` output assets is not available, maybe because the job ran on old version runtime, trying to get `flow_outputs` output asset instead.")
        output_asset_id = get_output_asset_id(run_id, "flow_outputs")
        relative_path = get_asset_path(output_asset_id)
        return relative_path.replace("flow_outputs", "flow_artifacts")

# Download flow artifacts for the given run_id
def download_flow_artifacts(run_id, blob_prefix):
    logger.info(f"Downloading Flow Artifacts for Run {run_id}")
    target_dir = f"./downloads/{run_id}"
    default_datastore.download(target_dir, prefix=blob_prefix, overwrite=True)
    return target_dir

# Load flow artifacts of the run from the given local_dir
def load_flow_artifacts(local_dir, run_display_name):
    logger.info(f"Loading Flow Artifacts of {run_display_name} from {local_dir}")
    flow_artifacts = {}
    p = Path(local_dir)
    for file_path in p.glob("**/*.jsonl"):
        with open(file_path) as fp:
            for line in fp:
                line_record = json.loads(line)
                line_number = line_record.get("line_number")
                run_info = line_record.get("run_info") or {}
                status = run_info.get("status")
                outputs = run_info.get("output") or {}
                inputs = run_info.get("inputs") or {}
                modified_inputs = {f"inputs.{k}": v for k, v in inputs.items()}
                record = {
                    "Line number": line_number,
                    "Run": run_display_name,
                    "Status": status,
                    **modified_inputs,
                    **outputs
                }
                flow_artifacts[line_number] = record
    return flow_artifacts

# Get required info for the given run_id
def get_required_info(run_id):
    logger.info(f"Processing Run: {run_id}")
    display_name, input_run_id = get_run_info(run_id)
    flow_artifact_relative_path = get_flow_artifact_relative_path(run_id)

    target_dir = download_flow_artifacts(run_id, flow_artifact_relative_path)
    flow_artifacts = load_flow_artifacts(target_dir, display_name)
    return {
        "run_id": run_id,
        "display_name": display_name,
        "input_run_id": input_run_id,
        "flow_artifacts": flow_artifacts
    }

# Update downstream flow artifacts with the given display_name
def update_downstread_flow_artifacts(flow_artifacts, display_name):
    updated_flow_artifacts = {}
    for line_number, line_record in flow_artifacts.items():
        updated_line_record = {}
        for k, v in line_record.items():
            if k == "Run" or k == "Status" or k == "Line number":
                continue
            else:
                updated_line_record[f"{k}({display_name})"] = v
        updated_flow_artifacts[line_number] = updated_line_record
    return updated_flow_artifacts

# Merge flow artifacts for the given run_infos, joining flow artifacts with the same line number
def merge_flow_artifacts(run_infos: list):
    run_info_dict = {run_info["run_id"]: run_info for run_info in run_infos}
    main_flow_infos = []
    for run_id, run_info in run_info_dict.items():
        if run_info["input_run_id"] is None:
            main_flow_infos.append(run_info)
        else:
            input_run_id = run_info["input_run_id"]
            if input_run_id in run_info_dict:
                input_run_info = run_info_dict[input_run_id]
                updated_flow_artifacts = update_downstread_flow_artifacts(run_info["flow_artifacts"], run_info["display_name"])
                for line_number, updated_flow_artifact in updated_flow_artifacts.items():
                    if line_number in input_run_info["flow_artifacts"]:
                        input_run_info["flow_artifacts"][line_number].update(updated_flow_artifact)
            else:
                # Input Run is not included, treat this run as main flow
                main_flow_infos.append(run_info)
    
    merge_result = []
    for main_flow_info in main_flow_infos:
        merge_result += list(main_flow_info["flow_artifacts"].values())
    merge_result = sorted(merge_result, key=lambda x: f"{x['Line number']}{x['Run']}")
    return merge_result

# Convert the given jsonl file to csv file
def jsonl_to_csv(jsonl_path, csv_path):
    max_keys = []
    with open(jsonl_path, "r",encoding='utf-8') as jsonl_file:
        for line in jsonl_file:
            row = json.loads(line)
            if len(row.keys()) > len(max_keys):
                max_keys = list(row.keys())

    with open(jsonl_path, "r",encoding='utf-8') as jsonl_file:
        with open(csv_path, "w", newline="",encoding='utf-8') as csv_file:
            writer = csv.writer(csv_file)
            writer.writerow(max_keys)
            for line in jsonl_file:
                row = json.loads(line)
                csv_row = [row.get(key) for key in max_keys]
                writer.writerow(csv_row)

## Download runs
Download jsonl of the runs, merge them and save to local jsonl_path and csv_path.

In [4]:
# The run_ids include all the runs selected from the UI.
run_ids = ["bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833"]

# Download the flow artifacts from the run_ids and merge them into a single file.
deduped_run_ids = list(set(run_ids))
run_infos = [get_required_info(run_id) for run_id in deduped_run_ids]
merged_result = merge_flow_artifacts(run_infos)

# Save the merged result as jsonl and csv.
jsonl_path = "./merged_result.jsonl"
csv_path = "./merged_result.csv"
with open(jsonl_path, "w") as fp:
    for record in merged_result:
        fp.write(json.dumps(record))
        fp.write("\n")
logger.info(f"Saved merged result as jsonl to {jsonl_path}")
jsonl_to_csv(jsonl_path, csv_path)
logger.info(f"Saved merged result as csv to {csv_path}")

2024-05-07 11:33:05,082 Processing Run: bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833
2024-05-07 11:33:05,082 Getting Run Info for Run: bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833
2024-05-07 11:33:18,003 Getting Flow Artifact Relative Path for Run bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833
2024-05-07 11:33:18,003 Getting Output Asset Id for Run bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833
2024-05-07 11:33:22,745 Getting Asset Path for Asset Id azureml://locations/eastus2/workspaces/c44e9724-0e65-4fa4-9b62-ebc0f390b962/data/azureml_bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833_output_data_debug_info/versions/1
2024-05-07 11:33:27,576 Downloading Flow Artifacts for Run bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833


Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000000150_000000174.jsonl, 1 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000000000_000000024.jsonl, 2 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000000100_000000124.jsonl, 3 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000000300_000000324.jsonl, 4 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000000075_000000099.jsonl, 5 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000000175_000000199.jsonl, 6 files out of an estimated total of 680
Downloaded promptflow/Prompt

Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000000700_000000724.jsonl, 30 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000000775_000000799.jsonl, 31 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000000625_000000649.jsonl, 32 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000000800_000000824.jsonl, 33 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000000875_000000899.jsonl, 34 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000000825_000000849.jsonl, 35 files out of an estimated total of 680
Downloaded promptflow/

Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000001475_000001499.jsonl, 59 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000001325_000001349.jsonl, 60 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000001650_000001674.jsonl, 61 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000001500_000001524.jsonl, 62 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000001575_000001599.jsonl, 63 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000001600_000001624.jsonl, 64 files out of an estimated total of 680
Downloaded promptflow/

Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000002100_000002124.jsonl, 88 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000002225_000002249.jsonl, 89 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000002150_000002174.jsonl, 90 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000002300_000002324.jsonl, 91 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000002475_000002499.jsonl, 92 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000002375_000002399.jsonl, 93 files out of an estimated total of 680
Downloaded promptflow/

...

Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000016000_000016024.jsonl, 581 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000016100_000016124.jsonl, 582 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000016075_000016099.jsonl, 583 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000016050_000016074.jsonl, 584 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000016150_000016174.jsonl, 585 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000016025_000016049.jsonl, 586 files out of an estimated total of 680
Downloaded promp

Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000016750_000016774.jsonl, 610 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000016675_000016699.jsonl, 611 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000016900_000016924.jsonl, 612 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000016800_000016824.jsonl, 613 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000016875_000016899.jsonl, 614 files out of an estimated total of 680
Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000016925_000016949.jsonl, 615 files out of an estimated total of 680
Downloaded promp

2024-05-07 11:53:36,211 Loading Flow Artifacts of evaluation_2 from ./downloads/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833


Downloaded promptflow/PromptFlowArtifacts/bb9f76b3-2e4f-4a38-ad44-7d3d93fc6833/flow_artifacts/000016850_000016874.jsonl, 620 files out of an estimated total of 680


2024-05-07 11:54:04,953 Saved merged result as jsonl to ./merged_result.jsonl
2024-05-07 11:54:11,554 Saved merged result as csv to ./merged_result.csv


## Visualize data
Load local csv file and visualize it as pandas dataframe. Make sure you have installed pandas:

    pip install pandas

In [5]:
# Load the merged result as pandas dataframe and sort it by line number.
import pandas as pd
data = pd.read_csv('merged_result.csv', dtype={'Line number': 'int64'})
df = pd.DataFrame(data)
df.sort_values(by='Line number', ascending=True)

,Line number,Run,Status,inputs.question,inputs.answer,inputs.context,inputs.ground_truth,inputs.metrics,inputs.threshold,inputs.groundedness_service_flight,...,gpt_groundedness_reason,gpt_similarity,gpt_fluency,sexual,self_harm_score,hate_unfairness_reason,violence,hate_unfairness_score,self_harm_reason,f1_score
0,0,evaluation_2,Completed,Many people have car where they live. The thin...,Many people have car where they live. The thin...,NaN,NaN,"gpt_coherence,gpt_fluency",4,True,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8007,1,evaluation_2,Completed,I am a scientist at NASA that is discussing th...,I am a scientist at NASA that is discussing th...,NaN,NaN,"gpt_coherence,gpt_fluency",4,True,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9118,2,evaluation_2,Completed,People always wish they had the same technolog...,People always wish they had the same technolog...,NaN,NaN,"gpt_coherence,gpt_fluency",4,True,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10229,3,evaluation_2,Completed,"We all heard about Venus, the planet without a...","We all heard about Venus, the planet without a...",NaN,NaN,"gpt_coherence,gpt_fluency",4,True,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11340,4,evaluation_2,Completed,"Dear, State Senator\n\nThis is a letter to arg...","Dear, State Senator\n\nThis is a letter to arg...",NaN,NaN,"gpt_coherence,gpt_fluency",4,True,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7666,16991,evaluation_2,Completed,"Many people throughout the world, would agree ...","Many people throughout the world, would agree ...",NaN,NaN,"gpt_coherence,gpt_fluency",4,True,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7667,16992,evaluation_2,Completed,Making technology that can identify human emot...,Making technology that can identify human emot...,NaN,NaN,"gpt_coherence,gpt_fluency",4,True,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7668,16993,evaluation_2,Completed,When election day comes and you finally drop y...,When election day comes and you finally drop y...,NaN,NaN,"gpt_coherence,gpt_fluency",4,True,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7669,16994,evaluation_2,Completed,The electoral collage is horable because the b...,The electoral collage is horable because the b...,NaN,NaN,"gpt_coherence,gpt_fluency",4,True,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Potential Problems and Solutions
  
### 1. [Winerror] File name or path too long/The system cannot find the path specified 

**Solution: Enable NTFS long paths in Windows**

**For Windows Home Users: Filesystem Registry entry**
1. Press `Win + R` keys on your keyboard and type `regedit` then press `Enter`. The Registry Editor will be opened.
2. Go to `HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\FileSystem`.
3. Create a new 32-bit DWORD value named `LongPathsEnabled` and set it to `1`.
4. Restart your Windows.

**For Windows Pro and Enterprise (Windows Server 2016 or above) Users: Using Group Policy**
1. Press `Win + R` keys on your keyboard and type `gpedit.msc` then press `Enter`. The Group Policy Editor will be opened.
2. Go to `Local Computer Policy -> Computer Configuration -> Administrative Templates -> System -> Filesystem`, then enable the `Enable Win32 long paths` option.
3. Restart your Windows.